In [ ]:
# https://github.com/evanm31/p4k-scraper/blob/master/data/scrapefork.py

In [ ]:
# !python -m pip install retry
# !python -m pip install backoff

Import packages and set uip enviornment

In [ ]:
import urllib
import re
import time
import retry
import backoff
import pandas as pd
import requests
import datetime
import os
from bs4 import BeautifulSoup

import pitchfork_ratings_def # my custom functions from github

if os.name == "posix":
    data_path = "/Users/kylezengo/Desktop/DS/Music Ratings/"

Load historic data

In [ ]:
# Load list of album review links
links_file_list = os.listdir(data_path+"links")

links_list = []
for i in links_file_list:
    links = open(data_path+"links/"+i)
    links = links.read().splitlines()
    
    links_list += links

links_list = list(set(links_list))
links_list = [x for x in links_list if x.startswith("https")]
print(f'{len(links_list)} album review links')

In [ ]:
# # Not sure what to do with this yet...
# pd.read_csv(data_path+"reviews/pitchfork_reviews_hist.csv").head()

In [ ]:
reviews_file_list = os.listdir(data_path+"reviews")
reviews_file_list.remove('pitchfork_reviews_hist.csv')

reviews = []
for i in reviews_file_list:
    df = pd.read_csv(data_path+"reviews/"+i)
    reviews.append(df)

reviews = pd.concat(reviews).drop_duplicates().reset_index(drop=True)
print(f'{reviews.shape[0]} album reviews in data frame')
reviews.head()

Get the link of each album review and save off to text file

In [ ]:
%%time
# 3-4s per page (5-10min to run)

new_links = []
i = 1
while requests.get("https://pitchfork.com/reviews/albums/?page="+str(i)).status_code==200:
    req = urllib.request.Request("https://pitchfork.com/reviews/albums/?page=" + str(i))
    html_page = pitchfork_ratings_def.urlopen_with_retry(req)
    print(f'{i}: html_page status code {html_page.getcode()}')

    soup = BeautifulSoup(html_page, "lxml")
    
    for link in soup.findAll('a'):
        link_get_href = link.get('href')
        if link_get_href == '/reviews/albums/':
            pass
        elif link_get_href.startswith( '/reviews/albums/?genre=' ):
            pass
        elif link_get_href.startswith( '/reviews/albums/' ):
            new_links.append("https://pitchfork.com"+link_get_href)
    i+=1

new_links = list(set(new_links))

In [ ]:
len(new_links)

In [ ]:
# Save list since takes time to generate
with open(f'{data_path}links/new_links_{datetime.datetime.now().date()}.txt', 'w') as f:
    for item in new_links:
        f.write("%s\n" % item)

Scrape reviews for each album and save to csv

In [ ]:
links_list_with_new = links_list+new_links
    
checked_links = set(df['link'])
not_checked_links = [x for x in links_list_with_new if x not in checked_links]
print(f'{len(not_checked_links)} links without review data')

In [ ]:
# # test links
# new_links = ["https://pitchfork.com/reviews/albums/james-devane-searching/",
#              "https://pitchfork.com/reviews/albums/blood-incantation-absolute-elsewhere/",
#              "https://pitchfork.com/reviews/albums/body-meat-starchris/",
#              "https://pitchfork.com/reviews/albums/fred-again-actual-life-3-january-1-september-9-2022/",
#              "https://pitchfork.com/reviews/albums/100-gecs-10000-gecs/"]

In [ ]:
%%time
# 2s? per page (long time to run >1 hr)

dat = []
for idx, i in enumerate(not_checked_links):
    print(f'{idx}: {i}')
    dat.append(pitchfork_ratings_def.gather_info(i))

new_reviews = pd.concat(dat).reset_index(drop=True)

In [ ]:
new_reviews = pd.concat(dat).reset_index(drop=True)

In [ ]:
new_reviews

In [ ]:
# # Testing errors
# album_link = "https://pitchfork.com/reviews/albums/various-artists-even-the-forest-hums-ukrainian-sonic-archives/" 
# gather_info(album_link)

# page = requests.get(album_link) #request URL
# soup = BeautifulSoup(page.content, 'html.parser') #parse with beautifulsoup
# soup

In [ ]:
new_reviews.to_csv(f'{data_path}reviews/new_reviews_{datetime.datetime.now().date()}.csv',index=False)